In [ ]:
from crewai import Agent, Task, Crew
from langchain.tools import tool
from langchain.llms import OpenAI
import os
import json

# Configuration de base
PDF_EXTRACT_PATH = "textes_extraits/sdata201635_pages_125-132.txt"  # Chemin vers votre fichier de texte extrait

# Définir les outils personnalisés
@tool
def lire_fichier_texte(chemin_fichier: str) -> str:
    """Lit un fichier texte et retourne son contenu"""
    try:
        with open(chemin_fichier, 'r', encoding='utf-8') as fichier:
            return fichier.read()
    except Exception as e:
        return f"Erreur lors de la lecture du fichier: {str(e)}"

@tool
def ecrire_resultats(contenu: str, nom_fichier: str = "resultats.txt") -> str:
    """Écrit les résultats dans un fichier"""
    try:
        with open(nom_fichier, 'w', encoding='utf-8') as fichier:
            fichier.write(contenu)
        return f"Résultats écrits avec succès dans {nom_fichier}"
    except Exception as e:
        return f"Erreur lors de l'écriture des résultats: {str(e)}"

# Définir les agents
agent_extraction = Agent(
    role="Analyste de Documents",
    goal="Analyser le contenu du fichier texte extrait et en extraire les informations pertinentes",
    backstory="""Vous êtes un expert en analyse de texte scientifique. 
    Votre spécialité est d'identifier les informations clés dans des documents 
    complexes et de les structurer de manière compréhensible.""",
    verbose=True,
    tools=[lire_fichier_texte],
    llm=OpenAI(temperature=0.7)
)

agent_presentation = Agent(
    role="Présentateur de Données",
    goal="Présenter les résultats d'analyse de manière claire et structurée",
    backstory="""Vous êtes un expert en communication scientifique.
    Votre talent est de transformer des données complexes en présentations 
    claires et informatives adaptées à différents publics.""",
    verbose=True,
    tools=[ecrire_resultats],
    llm=OpenAI(temperature=0.3)
)

# Définir les tâches
tache_extraction = Task(
    description=f"""
    Analysez le contenu du fichier texte extrait situé à {PDF_EXTRACT_PATH}.
    Identifiez les informations suivantes (à titre d'exemple, à remplacer par vos besoins spécifiques):
    1. Le sujet principal du document
    2. Les données numériques importantes
    3. Les méthodologies mentionnées
    4. Les conclusions principales
    
    Structurez ces informations dans un format JSON pour faciliter leur traitement ultérieur.
    """,
    agent=agent_extraction,
    expected_output="Un dictionnaire JSON contenant les informations extraites du document"
)

tache_presentation = Task(
    description="""
    En utilisant les informations extraites par l'Analyste de Documents,
    créez une présentation structurée des résultats qui inclut:
    1. Un résumé exécutif
    2. Les points clés organisés par catégorie
    3. Une conclusion avec recommandations
    
    Le format doit être professionnel et facile à lire. Utilisez des sections 
    clairement délimitées et une structure hiérarchique.
    """,
    agent=agent_presentation,
    expected_output="Un rapport structuré présentant les résultats de l'analyse"
)

# Configurer l'équipe (crew)
crew = Crew(
    agents=[agent_extraction, agent_presentation],
    tasks=[tache_extraction, tache_presentation],
    verbose=2  # Niveau de détail des logs
)

# Exécuter l'équipe
result = crew.kickoff()

print("\n======== RÉSULTAT FINAL ========")
print(result)